In [4]:
import os
import re
import datetime
import numpy
import pickle
from pprint import pformat
os.environ["DJANGO_SETTINGS_MODULE"] = "analyzer.settings"
import django
django.setup()

from analyzer import settings
from record.models import User, Group, Summary

username_expr = re.compile('(?P<season>[1-3])teku(?P<id>[0-9]+)')

datasets = {1: [], 2: [], 3: []}
seasons = {
    1: datetime.datetime.strptime('2015/09/11', '%Y/%m/%d'),
    2: datetime.datetime.strptime('2015/10/06', '%Y/%m/%d'),
    3: datetime.datetime.strptime('2015/11/01', '%Y/%m/%d')
}
for user in User.objects.all():
    m = username_expr.match(user.username)
    if not m:
        continue
    season = int(m.group('season'))
    season_start_date = seasons[season]
    date_dict = dict((season_start_date + datetime.timedelta(days=i), 0) for i in range(10))
    for summary in Summary.objects.filter(username=user.username):
        if summary.steps == 0: continue
        date_dict[summary.date] += summary.distance
    ary = [distance for date, distance in sorted(date_dict.items(), key=lambda x: x[0])]
    if sum(ary) == 0:
        continue
    datasets[season].append(ary)

for k, v in datasets.items():
    with open('10days_distance.seasion{}.pkl'.format(k), 'wb') as fp:
        pickle.dump(numpy.array(v), fp)